In [10]:
import pandas as pd       
import requests        
from bs4 import BeautifulSoup         
import os      
import numpy as np     
import pandas as pd      
import yfinance as yf
from io import StringIO

In [11]:
def fetch_info():  
    try:      
        url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
        headers = { 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0', 
            'Accept': 'application/json',
            'Accept-Language': 'en-US,en;q=0.5',
        }

                
        #  Send GET request
        response = requests.get(url, headers=headers)


        soup = BeautifulSoup(response.content, "html.parser")

        #  Get the symbols table
        tables = soup.find_all('table')
        #  #  Convert table to dataframe
        df = pd.read_html(StringIO(str(tables)))[2]
        #  Cleanup
        df.drop(columns=['Notes'], inplace=True)       
        return df    
    except Exception as e:   
        print(f'Error loading data  {e}')
        return None
    
dji_df = fetch_info()
print(dji_df)

               Company Exchange Symbol                        Industry  \
0                   3M     NYSE    MMM                    Conglomerate   
1     American Express     NYSE    AXP              Financial services   
2                Amgen   NASDAQ   AMGN               Biopharmaceutical   
3               Amazon   NASDAQ   AMZN                       Retailing   
4                Apple   NASDAQ   AAPL          Information technology   
5               Boeing     NYSE     BA           Aerospace and defense   
6          Caterpillar     NYSE    CAT         Construction and mining   
7              Chevron     NYSE    CVX              Petroleum industry   
8                Cisco   NASDAQ   CSCO          Information technology   
9            Coca-Cola     NYSE     KO                  Drink industry   
10              Disney     NYSE    DIS  Broadcasting and entertainment   
11                 Dow     NYSE    DOW               Chemical industry   
12       Goldman Sachs     NYSE     GS

### Tickers

In [12]:
tickers = dji_df.Symbol.values.tolist()
print(tickers)

['MMM', 'AXP', 'AMGN', 'AMZN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WMT']


### Download Tickers 

In [13]:
start_date = "2021-01-01"
end_date = "2022-09-01"
df = yf.download(tickers, start=start_date, end=end_date)
df.head

[*********************100%***********************]  30 of 30 completed


<bound method NDFrame.head of Price                       Adj Close                                      \
Ticker                           AAPL        AMGN        AMZN         AXP   
Date                                                                        
2021-01-04 00:00:00+00:00  126.683434  201.544556  159.331497  112.164017   
2021-01-05 00:00:00+00:00  128.249741  202.522675  160.925507  112.762657   
2021-01-06 00:00:00+00:00  123.932648  207.404297  156.919006  116.934120   
2021-01-07 00:00:00+00:00  128.161606  208.089035  158.108002  116.009186   
2021-01-08 00:00:00+00:00  129.267807  212.063690  159.134995  116.123596   
...                               ...         ...         ...         ...   
2022-08-25 00:00:00+00:00  168.166748  230.739624  137.279999  157.373306   
2022-08-26 00:00:00+00:00  161.826981  226.015488  130.750000  152.600616   
2022-08-29 00:00:00+00:00  159.611526  224.822739  129.789993  149.913513   
2022-08-30 00:00:00+00:00  157.168594  224.578

In [14]:
df.columns

MultiIndex([('Adj Close', 'AAPL'),
            ('Adj Close', 'AMGN'),
            ('Adj Close', 'AMZN'),
            ('Adj Close',  'AXP'),
            ('Adj Close',   'BA'),
            ('Adj Close',  'CAT'),
            ('Adj Close',  'CRM'),
            ('Adj Close', 'CSCO'),
            ('Adj Close',  'CVX'),
            ('Adj Close',  'DIS'),
            ...
            (   'Volume',  'MMM'),
            (   'Volume',  'MRK'),
            (   'Volume', 'MSFT'),
            (   'Volume',  'NKE'),
            (   'Volume',   'PG'),
            (   'Volume',  'TRV'),
            (   'Volume',  'UNH'),
            (   'Volume',    'V'),
            (   'Volume',   'VZ'),
            (   'Volume',  'WMT')],
           names=['Price', 'Ticker'], length=180)

In [15]:
df = df['Adj Close']
df.head

<bound method NDFrame.head of Ticker                           AAPL        AMGN        AMZN         AXP  \
Date                                                                        
2021-01-04 00:00:00+00:00  126.683434  201.544556  159.331497  112.164017   
2021-01-05 00:00:00+00:00  128.249741  202.522675  160.925507  112.762657   
2021-01-06 00:00:00+00:00  123.932648  207.404297  156.919006  116.934120   
2021-01-07 00:00:00+00:00  128.161606  208.089035  158.108002  116.009186   
2021-01-08 00:00:00+00:00  129.267807  212.063690  159.134995  116.123596   
...                               ...         ...         ...         ...   
2022-08-25 00:00:00+00:00  168.166748  230.739624  137.279999  157.373306   
2022-08-26 00:00:00+00:00  161.826981  226.015488  130.750000  152.600616   
2022-08-29 00:00:00+00:00  159.611526  224.822739  129.789993  149.913513   
2022-08-30 00:00:00+00:00  157.168594  224.578522  128.729996  150.029938   
2022-08-31 00:00:00+00:00  155.497116  225.686

### monthly return

In [16]:
mth_return_df = df.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)
print(mth_return_df)

Ticker                         AAPL      AMGN      AMZN       AXP        BA  \
Date                                                                          
2021-01-31 00:00:00+00:00  0.019705  0.065163  0.006141 -0.011626 -0.042078   
2021-02-28 00:00:00+00:00 -0.079712 -0.061457 -0.035328  0.163427  0.091766   
2021-03-31 00:00:00+00:00  0.007340  0.106215  0.000372  0.048833  0.201453   
2021-04-30 00:00:00+00:00  0.076218 -0.036855  0.120663  0.084206 -0.080127   
2021-05-31 00:00:00+00:00 -0.050497 -0.000113 -0.070470  0.044213  0.054244   
2021-06-30 00:00:00+00:00  0.099109  0.024418  0.067355  0.031849 -0.030200   
2021-07-31 00:00:00+00:00  0.064982 -0.009067 -0.032722  0.034770 -0.054600   
2021-08-31 00:00:00+00:00  0.042489 -0.059073  0.043034 -0.026799 -0.030819   
2021-09-30 00:00:00+00:00 -0.068036 -0.057110 -0.053518  0.009460  0.002005   
2021-10-31 00:00:00+00:00  0.058657 -0.026710  0.026602  0.039885 -0.058698   
2021-11-30 00:00:00+00:00  0.105082 -0.031020  0.039

In [17]:
# obtain the historical cumulative returns of past 6 months as the terminal return of current month
past_cum_return_df = (mth_return_df+1).rolling(6).apply(np.prod) - 1
print(past_cum_return_df)

Ticker                         AAPL      AMGN      AMZN       AXP        BA  \
Date                                                                          
2021-01-31 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2021-02-28 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2021-03-31 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2021-04-30 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2021-05-31 00:00:00+00:00       NaN       NaN       NaN       NaN       NaN   
2021-06-30 00:00:00+00:00  0.061723  0.091013  0.079561  0.408910  0.181728   
2021-07-31 00:00:00+00:00  0.108867  0.014981  0.037861  0.475046  0.166280   
2021-08-31 00:00:00+00:00  0.256109  0.017560  0.122169  0.233869  0.035329   
2021-09-30 00:00:00+00:00  0.162118 -0.132676  0.061718  0.187551 -0.136542   
2021-10-31 00:00:00+00:00  0.143156 -0.123540 -0.027395  0.139006 -0.116427   
2021-11-30 00:00:00+00:00  0.330465 -0.150633  0.088

In [18]:
import datetime as dt
                
end_of_measurement_period = dt.datetime(2022,6,30,tzinfo=dt.timezone.utc)
print(end_of_measurement_period)

formation_period = dt.datetime(2022,7,31,tzinfo=dt.timezone.utc)

end_of_measurement_period_return_df = past_cum_return_df.loc[end_of_measurement_period]

end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()

print(end_of_measurement_period_return_df)
print('max')
print( end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()])
print(end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()])

2022-06-30 00:00:00+00:00
   Ticker  2022-06-30 00:00:00+00:00
0    AAPL                  -0.227936
1    AMGN                   0.099514
2    AMZN                  -0.362932
3     AXP                  -0.144964
4      BA                  -0.320882
5     CAT                  -0.126977
6     CRM                  -0.350569
7    CSCO                  -0.318526
8     CVX                   0.256954
9     DIS                  -0.390535
10    DOW                  -0.069513
11     GS                  -0.214219
12     HD                  -0.330937
13    HON                  -0.157529
14    IBM                   0.082029
15   INTC                  -0.262273
16    JNJ                   0.051017
17    JPM                  -0.279288
18     KO                   0.078358
19    MCD                  -0.068791
20    MMM                  -0.256987
21    MRK                   0.209992
22   MSFT                  -0.232991
23    NKE                  -0.383890
24     PG                  -0.111276
25    TRV   

### Divide into 5 group

In [19]:
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 5, labels=False)
end_of_measurement_period_return_df

,Ticker,2022-06-30 00:00:00+00:00,rank
0,AAPL,-0.227936,1
1,AMGN,0.099514,4
2,AMZN,-0.362932,0
3,AXP,-0.144964,2
4,BA,-0.320882,0
5,CAT,-0.126977,2
6,CRM,-0.350569,0
7,CSCO,-0.318526,1
8,CVX,0.256954,4
9,DIS,-0.390535,0


### long the stock when rank = 4 (MAX) Short the stock when rank =0 

In [24]:
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==4,'Ticker'].values
print(long_stocks)
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,'Ticker'].values
print(short_stocks)

['AMGN' 'CVX' 'IBM' 'KO' 'MRK' 'TRV']
['AMZN' 'BA' 'CRM' 'DIS' 'HD' 'NKE']


### Evaluating Out-of-Sample Performance

In [ ]:
from dateutil.relativedelta import relativedelta

print(formation_period)

long_return_df = mth_return_df.loc[formation_period +  relativedelta(months=1),  mth_return_df.columns.isin(long_stocks)]
print(long_return_df)

short_return_df = mth_return_df.loc[formation_period +  relativedelta(months=1),  mth_return_df.columns.isin(short_stocks)]
print(short_return_df)

2022-07-31 00:00:00+00:00
Ticker
AMGN   -0.021474
CVX    -0.026156
IBM    -0.005517
KO     -0.038336
MRK    -0.044549
TRV     0.018525
Name: 2022-08-31 00:00:00+00:00, dtype: float64
Ticker
AMZN   -0.060615
BA      0.005900
CRM    -0.151614
DIS     0.056362
HD     -0.035350
NKE    -0.073704
Name: 2022-08-31 00:00:00+00:00, dtype: float64


### profit

In [30]:
momentum_profit = long_return_df.mean() - short_return_df.mean()
print(momentum_profit)

0.023585449040311107
